# tRIBS-Sandbox: Run Model

The goal for this notebook is to first run the model we made then use pytRIBS to load in our model outputs and make some plots from the model outputs. The plots shown below are just examples and each output file has multiple variables that can be plotted. Addtion detail on the available model outputs is located [Here](https://tribshms.readthedocs.io/en/latest/man/Output.html)

## Imports

In [ ]:
# note you can install pytRIBS via pip; see: https://pypi.org/project/pytRIBS/
from pytRIBS.classes import *

In [ ]:
# if you have installed pytRIBS, the following libraries should already be in your environment
import os, sys, shutil
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from shapely.ops import unary_union
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors

### Run The Model
In our previous notebook we got everything organized to run the model. Typically tRIBS is ran from the command line but in this codespace we setup and direct path to the tRIBS executeable we can access from within this notebook. So lets run the simulation:

In [ ]:
import time

# Define File Names
input_filename = 'SMF.in'  
log_filename   = 'simulation.log'

# Pre-run check
# Check if the input file actually exists before trying to run.
if not os.path.exists(input_filename):
    print(f"ERROR: Could not find input file: '{input_filename}'")
    print("Please check your spelling or ensure the previous pytRIBS step ran correctly.")
else:
    print(f"Found input file: {input_filename}")
    print(f"Starting tRIBS simulation...")
    print(f"Runtime output is being redirected to: {log_filename}")

    # Execute model
    # We use the 'time' module to track how long the simulation takes.
    start_time = time.time()
    
    # Run tRIBS! 
    # syntax: !tRIBS <input_file> > <log_file> 2>&1
    exit_code = os.system(f"tRIBS {input_filename} > {log_filename} 2>&1")
    
    end_time = time.time()
    duration = (end_time - start_time) / 60

    # Post-run report
    if exit_code == 0:
        print(f"\nSUCCESS: Simulation completed in {duration:.2f} minutes.")

## Results Class: Merge and Visualize Results
The Results class simplifies working with tRIBS outputs by offering post-processing methods that handle everything from file management to basic model output analysis. tRIBS generates a large amount of data with fine spatial and temporal resolutions, including time series of streamflow and spatially averaged state and flux variables. Additionally, the model produces Voronoi diagrams that can be used with both dynamic snapshots (captured at specific times) and integrated outputs (aggregated over the entire model run). The Results class helps manage these outputs, providing users with tools to merge parallel results and perform further analysis using commonly utilized data libraries.

In [ ]:
name ='SMF'
epsg = 26912
proj = Project(os.getcwd(),name,epsg) 
# With this command below pytRIBS will read the input file to ge tthe filepaths of all output locations
results = Results('SMF.in',meta=proj.meta)

In [ ]:
gdf = results.voronoi.merge(results.int_spatial_vars,on='ID')
results.get_mrf_results()
results.get_element_results()

strmflw_sim_raw = results.get_qout_results()
mrf = results.mrf['mrf']
mrf.set_index('Time',inplace=True)

### Visualize Results
First lets start with plotting the tRIBS outlet streamflow timeseries and compare it to the observations. Our ouputs and obervational data are not in the same time intervals so we need to do some preprocessing first.

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# LOAD AND ORGANIZE OBSERVED DATA
# Load data from Excel file
obs_filepath = '../smf_init_data/met/SMF_Observations_1993-2025.xlsx'
obs_df = pd.read_excel(obs_filepath, sheet_name='Discharge', skiprows=6)

# Organize observation so that they are datetime dataframe
obs_df['datetime'] = pd.to_datetime(obs_df['Date'].astype(str) + ' ' + obs_df['Time'].astype(str))

# Set the datetime as the index 
obs_df.set_index('datetime', inplace=True)

# Convert CFS to CMS (m^3/s) to match tRIBS output. 
obs_df['Observed_CMS'] = obs_df['cfs'] * 0.0283168


# PREPARE SIMULATED DATA TIMESERIES
# All we need to do is convert our dataframe into having a datetime index
strmflw_sim = strmflw_sim_raw
strmflw_sim['Time'] = pd.to_datetime(strmflw_sim['Time'])
strmflw_sim.set_index('Time', inplace=True)


# RESAMPLE AND ALIGN DATA
# tRIBS output is every 3.75 mins. Observed is the raw reports.
# We resample both to a common frequency. Let's use 5 minute averages ('5T') averages. 
obs_resampled = obs_df['Observed_CMS'].resample('5T').mean()
sim_resampled = strmflw_sim['Qstrm_m3s'].resample('5T').mean()

# Combine into a single dataframe and drop rows where we don't have both data points.
compare_df = pd.DataFrame({
    'Observed': obs_resampled,
    'Simulated': sim_resampled
}).dropna()

# Define the start and end of the storm event you want to focus on. so we can clip the dataframe
event_start = '2014-08-12 16:00'
event_end   = '2014-08-12 23:00'
event_df = compare_df.loc[event_start:event_end]


# PLOTTING THE HYDROGRAPHS
fig, ax = plt.subplots(figsize=(12, 6))

# Plot Simulated as a solid blue line
ax.plot(event_df.index, event_df['Simulated'], 
        label='Simulated (tRIBS)', color='#1f77b4', linewidth=2)
# Plot Observed as black dots/dashed line
ax.plot(event_df.index, event_df['Observed'], 
        label='Observed (Gauge)', color='black', marker='o', markersize=4, linestyle='--', linewidth=1)

# Formatting the plot
ax.set_title('Simulated vs. Observed Streamflow at Watershed Outlet', fontsize=14, fontweight='bold')
ax.set_ylabel('Streamflow ($m^3/s$)', fontsize=12)
ax.set_xlabel('Date', fontsize=12)

# Format the x-axis 
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.xticks(rotation=45)

ax.legend(fontsize=12)
ax.grid(True, linestyle=':', alpha=0.7)

plt.tight_layout()
plt.show()

KeyError: 'Time'

Now that we have orgnaized and plotted our streamflow data we can look at some preformance metrics

In [ ]:
# Make sure we are using the clipped storm event dataframe!
obs = event_df['Observed']
sim = event_df['Simulated']

# =========================================================
# 1. PHYSICAL EVENT METRICS (Peaks, Timing, Volumes)
# =========================================================
# Peak Discharge (m^3/s)
obs_peak = obs.max()
sim_peak = sim.max()

# Time to Peak
obs_tpeak = obs.idxmax()
sim_tpeak = sim.idxmax()

# Total Volume (m^3)
# Flow is in m^3/s. Our dataframe time interval is whatever we set our resampling interval to earlier.
# Lets calculate the time interval from the datafram directly to compute the flow volume.
dt_seconds = (event_df.index[1] - event_df.index[0]).total_seconds()
obs_vol_m3 = obs.sum() * dt_seconds
sim_vol_m3 = sim.sum() * dt_seconds

# Volume Error (%)
vol_error_pct = ((sim_vol_m3 - obs_vol_m3) / obs_vol_m3) * 100

# Statistical Goodness-of-Fit Metrics
# Root Mean Square Error (RMSE)
rmse = np.sqrt(np.mean((sim - obs)**2))

# Nash-Sutcliffe Efficiency (NSE)
# NSE = 1 means perfect match. NSE < 0 means the mean of observed is a better predictor than the model.
nse = 1 - (np.sum((sim - obs)**2) / np.sum((obs - obs.mean())**2))

# Percent Bias (PBIAS)
# Positive values indicate overestimation bias, negative indicates underestimation bias.
pbias = 100 * (np.sum(sim - obs) / np.sum(obs))

# Print a Simple Report
print("--- HYDROLOGICAL EVENT METRICS ---")
print(f"Observed Peak Flow: {obs_peak:.2f} m^3/s  (at {obs_tpeak.strftime('%m-%d %H:%M')})")
print(f"Simulated Peak Flow:{sim_peak:.2f} m^3/s  (at {sim_tpeak.strftime('%m-%d %H:%M')})")
print(f"Peak Timing Error:  {(sim_tpeak - obs_tpeak).total_seconds() / 3600:.1f} hours\n")

print(f"Observed Volume:    {obs_vol_m3:,.0f} m^3")
print(f"Simulated Volume:   {sim_vol_m3:,.0f} m^3")
print(f"Volume Error:       {vol_error_pct:+.1f}%\n")

print("--- STATISTICAL PERFORMANCE ---")
print(f"RMSE:               {rmse:.2f} m^3/s")
print(f"NSE:                {nse:.3f}")
print(f"Percent Bias:       {pbias:+.1f}%")

tRIBS has many model outputs. One of those is spatial outputs. For this sandbox environment th emodle output the integrated spatial output file which has outputs like time invariant watershed properties but also cumulative outputs of flux variable like evapotranspiration.

First spatial plot we will make is the voronoi polygon elevation map:

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Left PLOT: Elevation and Mesh Structure ---
gdf.plot(ax=ax1, column='Z', cmap='terrain', 
         edgecolor='black', linewidth=0.2,
         legend=True, legend_kwds={'label': 'Elevation (m)', 'shrink': 0.8})

ax1.set_title('tRIBS Mesh & Elevation', fontsize=14, fontweight='bold')
ax1.set_xlabel('Longitude', fontsize=12)
ax1.set_ylabel('Latitude', fontsize=12)
ax1.grid(True, linestyle=':', alpha=0.5)

# rightPLOT: Polygon ID
gdf.plot(ax=ax2, column='ID', cmap='viridis', 
         edgecolor='none', 
         legend=True, legend_kwds={'label': 'Polygon ID (Routing Order)', 'shrink': 0.8})

ax2.set_title('Computational Routing Order (Polygon IDs)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Longitude', fontsize=12)
ax2.set_yticklabels([]) 
ax2.grid(True, linestyle=':', alpha=0.5)

plt.tight_layout()
plt.show()

Next we can plot the cumulative ET for the entire 480 hour simulaiton

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Left PLOT: ET
gdf.plot(ax=ax1, column='cET', cmap='YlOrBr', legend=True,
         edgecolor='none', 
         legend_kwds={'label': 'Cumulative ET (mm)', 'orientation': 'vertical'})

ax1.set_title('Spatial Distribution of Evapotranspiration', fontsize=14, fontweight='bold')
ax1.set_xlabel('Longitude', fontsize=12)
ax1.set_ylabel('Latitude', fontsize=12) 
ax1.grid(True, linestyle=':', alpha=0.5)

# rightPLOT: Depth to Bedrock
gdf.plot(ax=ax2, column='Bedrock_Depth_mm', cmap='gist_earth', 
         edgecolor='none', 
         legend=True, legend_kwds={'label': 'Depth to Bedrock (mm)', 'shrink': 0.8})

ax2.set_title('Spatial Distribution of Bedrock Depth', fontsize=14, fontweight='bold')
ax2.set_xlabel('Longitude', fontsize=12)
ax2.set_yticklabels([]) 
ax2.grid(True, linestyle=':', alpha=0.5)

plt.tight_layout()
plt.show()

Interestingly the model has some pretty large differences in cumulative ET. The plot of depth to bedrock was added as one way of getting to an explanation. Since the soils are so shallow on the mountainous slopes the vegetation in those areas has full access to any soil water available.

Another coomonly used model output is the mean response file (`*.mrf`). This output is a timeseries of many of the model's important variables. In the plot below we include the 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

fig, ax1 = plt.subplots(figsize=(12, 6))

# Bottom AXIS: Depth to Groundwater Table 
ax1.plot(mrf.index, mrf['MDGW'], color='#2ca02c', linewidth=2, label='Basin-Avg Groundwater Depth')

ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel('Depth to Groundwater Table (mm)', fontsize=12, color='#2ca02c')
ax1.tick_params(axis='y', labelcolor='#2ca02c')

# Lets invert the Y-axis so we can see the depth increase as the watertable is drawn down
ax1.invert_yaxis() 

# Top AXIS: Rainfall Hyetograph
ax2 = ax1.twinx()

ax2.bar(mrf.index, mrf['MAP'], width=0.02, color='#1f77b4', alpha=0.6, label='Basin-Avg Rainfall')

ax2.set_ylabel('Rainfall (mm/hr)', fontsize=12, color='#1f77b4')
ax2.tick_params(axis='y', labelcolor='#1f77b4')
ax2.invert_yaxis()

# Set limits so the rainfall bars only take up the top 30% of the plot
max_rain = mrf['MAP'].max()
ax2.set_ylim(max_rain * 3, 0) 

# Formatting
ax1.set_title('Basin-Averaged Depth to Groundwater Table', fontsize=14, fontweight='bold')
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
ax1.grid(True, linestyle=':', alpha=0.7)

# Combine legends from both axes
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='center right', fontsize=11)

plt.tight_layout()
plt.show()

Here we plotted to the groundwater depth from mrf file only the lower axis and mean areal precipitation on the top axis. We can see with the shallow depth to bedrock the soil is almost instantly saturated.